## Проверка сравнения 2х переводов

In [1]:
import pandas as pd
from scipy.stats import binomtest

In [5]:
data = pd.read_csv("/Users/n.valieva/Downloads/trans_comparison.csv")

In [6]:
data.head(3)

,id,text,translation1,translation2,result
0,406_18,"Exactly, she has to understand that life goes ...","Точно, она должна понять, что жизнь не огранич...","Именно, она должна понять, что жизнь не ограни...",1
1,73_3,I am still having bad flashbacks from War in I...,У меня до сих пор плохие воспоминания о войне ...,У меня все еще плохие флэшбеки от войны в Ирак...,1
2,587_10,I would look to talk to a therapist about thes...,Я бы посоветовал поговорить с терапевтом об эт...,Я бы посоветовал обратиться к терапевту с этим...,2


In [25]:
data.shape

(200, 5)

In [16]:
data_filtered = data[data.result != 3]

In [24]:
n = data_filtered.shape[0]
n

186

In [21]:
new_win = data[data.result == 2].shape[0]
new_win

114

In [22]:
result = binomtest(new_win, n=n, p=0.5, alternative='greater')

In [23]:
result.pvalue

np.float64(0.0012752596263399644)

## Проверка оценки переводов экспертами

In [3]:
import pandas as pd
from scipy.stats import mannwhitneyu

In [5]:
estim = pd.read_csv('/Users/n.valieva/Downloads/translation estimation - оценка перевода.csv')

In [6]:
estim.head()

,id,text,translation,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем)
0,812_19,I think that is truly when I started feeling d...,"Думаю, именно тогда я начал чувствовать себя п...",4,3,2
1,900_20,I am hearing that while you have thought about...,"Я слышу, что хотя ты уже думала об обращении з...",4,2,3
2,431_26,That's great! IT's also a good place because y...,"Это здорово! Это ещё и хорошее место, потому ч...",3,4,1
3,1121_19,"yeah, lool something like that ;). Glad your b...","да, лол примерно так ;). Рад, что тебе лучше",3,5,5
4,260_25,the only thing you can really do right now is ...,"единственное, что ты можешь сделать прямо сейч...",5,4,5


In [11]:
estim['overall'] =  (estim['оценка перевода (Иван)'] + estim['оценка перевода (Екатерина)'] + estim['оценка перевода (Артем)']) / 3
estim['overall'] = estim['overall'].astype(int)

In [17]:
estim.shape

(50, 7)

In [12]:
estim.head()

,id,text,translation,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем),overall
0,812_19,I think that is truly when I started feeling d...,"Думаю, именно тогда я начал чувствовать себя п...",4,3,2,3
1,900_20,I am hearing that while you have thought about...,"Я слышу, что хотя ты уже думала об обращении з...",4,2,3,3
2,431_26,That's great! IT's also a good place because y...,"Это здорово! Это ещё и хорошее место, потому ч...",3,4,1,2
3,1121_19,"yeah, lool something like that ;). Glad your b...","да, лол примерно так ;). Рад, что тебе лучше",3,5,5,4
4,260_25,the only thing you can really do right now is ...,"единственное, что ты можешь сделать прямо сейч...",5,4,5,4


In [13]:
perplexity_selected_ids = pd.read_csv("top25_bad_perplexity.csv").id
random_selected_ids = pd.read_csv("25_samples_from_all_set.csv").id

In [14]:
perplexity_selected = estim[estim.id.isin(perplexity_selected_ids)]
random_selected = estim[estim.id.isin(random_selected_ids)]

In [16]:
perplexity_selected.shape, random_selected.shape

((25, 7), (25, 7))

In [18]:
mannwhitneyu(perplexity_selected.overall, random_selected.overall)

MannwhitneyuResult(statistic=np.float64(231.0), pvalue=np.float64(0.0938000706682478))

## Проверка оценки переводов моделями

In [9]:
%load_ext autoreload
%autoreload 2 

In [1]:
import sys
import os.path as osp
import os

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
sys.path.append(current_dir + '/notebooks')
sys.path.append(current_dir)
sys.path.append(current_dir + '/data')
sys.path.append(current_dir + '/src')
sys.path.append(current_dir + '/src/utils')
sys.path.append(current_dir + '/src/utils')

In [64]:
import json
from pathlib import Path
from omegaconf import OmegaConf
from datasets import Dataset
from src.core.translate import Translator
from src.utils.schemas import TranslationEstimationResultSchema
import pandas as pd 
from scipy.stats import chisquare
from collections import defaultdict

def read_json(path):
    return json.load(Path(path).open())

def save_json(obj, path):
    json.dump(obj, Path(path).open("w", encoding="utf-8"), indent=4, ensure_ascii=False)
    
def read_file(path: str):
    	return Path(path).open().read()

In [57]:
estim = pd.read_csv('/Users/n.valieva/Downloads/translation estimation - оценка перевода.csv')

In [58]:
estim.head()

,id,text,translation,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем)
0,812_19,I think that is truly when I started feeling d...,"Думаю, именно тогда я начал чувствовать себя п...",4,3,2
1,900_20,I am hearing that while you have thought about...,"Я слышу, что хотя ты уже думала об обращении з...",4,2,3
2,431_26,That's great! IT's also a good place because y...,"Это здорово! Это ещё и хорошее место, потому ч...",3,4,1
3,1121_19,"yeah, lool something like that ;). Glad your b...","да, лол примерно так ;). Рад, что тебе лучше",3,5,5
4,260_25,the only thing you can really do right now is ...,"единственное, что ты можешь сделать прямо сейч...",5,4,5


In [59]:
dataset = Dataset.from_pandas(estim[['id', 'text', 'translation']]).rename_column("translation", "text_rus")

In [60]:
MODELS = ['gpt-4o', 'claude-sonnet-4', 'gemini-2.5-flash', 'qwen3-235b-a22b', 'deepseek-chat-v3-0324', 'llama-4-maverick']

In [ ]:
for model in MODELS:
	config = OmegaConf.load('../configs/conf.yaml')
	general_translation_config = config.general_translation

	general_translation_config.batch_result_dir = f"batches_res_general_translation_{model}"
	general_translation_config.batches = f"batches_general_translation_{model}"
	general_translation_config.batch_size = 10
	general_translation_config.prompt_path = "/Users/n.valieva/Desktop/empathy_dataset_transfer/configs/prompts/translation_estimation_prompt.txt"

	model_config = read_json(general_translation_config.model_config_path)
	model_config['model_name'] = model
	print(f"Using model: {model}")

	general_translator = Translator(
		system_message=read_file(general_translation_config.prompt_path), 
		model_config=model_config,
		example_data=read_json(general_translation_config.filepath_examples), 
		batch_size=general_translation_config.batch_size,
		batch_result_dir=general_translation_config.batch_result_dir,
		batch_dir=general_translation_config.batches,
		model_type="openai"
	)

	translation_result = general_translator.translate(dataset, TranslationEstimationResultSchema)

	save_json(translation_result, f"translation_result_estimation_{model}.json")

	estim = pd.DataFrame(translation_result).rename(columns={'value': f"value_{model}"}).merge(estim, on='id')

In [62]:
estim.head()

,id,value_llama-4-maverick,value_deepseek-chat-v3-0324,value_qwen3-235b-a22b,value_gemini-2.5-flash,value_claude-sonnet-4,value_gpt-4o,text,translation,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем)
0,812_19,5,5,5,5,5,5,I think that is truly when I started feeling d...,"Думаю, именно тогда я начал чувствовать себя п...",4,3,2
1,900_20,5,5,5,5,5,5,I am hearing that while you have thought about...,"Я слышу, что хотя ты уже думала об обращении з...",4,2,3
2,431_26,5,5,5,5,5,5,That's great! IT's also a good place because y...,"Это здорово! Это ещё и хорошее место, потому ч...",3,4,1
3,1121_19,4,5,5,4,5,4,"yeah, lool something like that ;). Glad your b...","да, лол примерно так ;). Рад, что тебе лучше",3,5,5
4,260_25,5,5,5,5,5,5,the only thing you can really do right now is ...,"единственное, что ты можешь сделать прямо сейч...",5,4,5


In [63]:
estim['overall_person'] = (estim['оценка перевода (Иван)'] + estim['оценка перевода (Екатерина)'] + estim['оценка перевода (Артем)']) / 3
estim['overall_person'] = estim['overall_person'].astype(int)

In [71]:
df_data = defaultdict(dict)

for model in MODELS:
	for col in ['оценка перевода (Иван)', 'оценка перевода (Екатерина)', 'оценка перевода (Артем)', 'overall_person']:
		expected = estim[col]
		observed = estim[f'value_{model}']
		expected_scaled = expected * (observed.sum() / expected.sum())
		df_data[model][col] = chisquare(f_obs=observed, f_exp=expected_scaled)[1]

In [72]:
pd.DataFrame(df_data).T

,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем),overall_person
gpt-4o,1.000000,0.175658,0.001253,0.999996
claude-sonnet-4,1.000000,0.387769,0.005886,1.000000
gemini-2.5-flash,0.999996,0.118529,0.000040,0.999956
qwen3-235b-a22b,0.999993,0.160640,0.000288,0.999976
deepseek-chat-v3-0324,0.999997,0.101087,0.000426,0.999987
llama-4-maverick,1.000000,0.214498,0.000771,0.999997


меняем (1, 2, 3) -> 1, (4, 5) -> 2

In [73]:
estim1 = pd.read_csv('/Users/n.valieva/Downloads/translation estimation - оценка перевода.csv')

In [ ]:
value_map = {1: 1, 2: 1, 3: 1, 4: 2, 5: 2}

In [76]:
for col in ['оценка перевода (Иван)', 'оценка перевода (Екатерина)', 'оценка перевода (Артем)']:
	estim1[col] = estim1[col].map(value_map)

In [78]:
estim1['overall_person'] = (estim1['оценка перевода (Иван)'] + estim1['оценка перевода (Екатерина)'] + estim1['оценка перевода (Артем)']) / 3
estim1['overall_person'] = estim1['overall_person'].astype(int)

In [79]:
dataset = Dataset.from_pandas(estim1[['id', 'text', 'translation']]).rename_column("translation", "text_rus")

In [88]:
examples_data = read_json(general_translation_config.filepath_examples)

In [90]:
for i, item in enumerate(examples_data['examples']):
    examples_data['examples'][i]['result']['value'] = value_map[item['result']['value']]

In [92]:
for model in MODELS:
    config = OmegaConf.load('../configs/conf.yaml')
    general_translation_config = config.general_translation
    general_translation_config.batch_result_dir = f"batches_res_general_translation_{model}_1"
    general_translation_config.batches = f"batches_general_translation_{model}_1"
    general_translation_config.batch_size = 10
    general_translation_config.prompt_path = "/Users/n.valieva/Desktop/empathy_dataset_transfer/configs/prompts/translation_estimation_prompt1.txt"
    model_config = read_json(general_translation_config.model_config_path)
    model_config['model_name'] = model
    print(f"Using model: {model}")
    general_translator = Translator(
		system_message=read_file(general_translation_config.prompt_path), 
		model_config=model_config,
		example_data=examples_data, 
		batch_size=general_translation_config.batch_size,
		batch_result_dir=general_translation_config.batch_result_dir,
		batch_dir=general_translation_config.batches,
		model_type="openai"
	)
    translation_result = general_translator.translate(dataset, TranslationEstimationResultSchema)
    save_json(translation_result, f"translation_result_estimation_{model}_1.json")
    estim1 = pd.DataFrame(translation_result).rename(columns={'value': f"value_{model}"}).merge(estim1, on='id')

Using model: gpt-4o


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 906.68ba/s]

Processing batch 1/5...



Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1792.44ba/s]


...

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 541.41ba/s]


Processing batch 4/5...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 580.69ba/s]


Processing batch 5/5...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 608.58ba/s]


In [103]:
df_data1 = defaultdict(dict)

for model in MODELS:
	for col in ['оценка перевода (Иван)', 'оценка перевода (Екатерина)', 'оценка перевода (Артем)', 'overall_person']:
		expected = estim1[col]
		observed = estim1[f'value_{model}']
		expected_scaled = expected * (observed.sum() / expected.sum())
		df_data1[model][col] = chisquare(f_obs=observed, f_exp=expected_scaled)[1]

In [104]:
df1 = pd.DataFrame(df_data1).rename(columns={col: f"{col}__12" for col in df_data1.keys()}).T

In [105]:
df = pd.DataFrame(df_data).rename(columns={col: f"{col}__15" for col in df_data.keys()}).T

In [106]:
df

,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем),overall_person
gpt-4o__15,1.000000,0.175658,0.001253,0.999996
claude-sonnet-4__15,1.000000,0.387769,0.005886,1.000000
gemini-2.5-flash__15,0.999996,0.118529,0.000040,0.999956
qwen3-235b-a22b__15,0.999993,0.160640,0.000288,0.999976
deepseek-chat-v3-0324__15,0.999997,0.101087,0.000426,0.999987
llama-4-maverick__15,1.000000,0.214498,0.000771,0.999997


In [107]:
df1

,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем),overall_person
gpt-4o__12,1.0,1.0,1.0,1.0
claude-sonnet-4__12,1.0,1.0,1.0,1.0
gemini-2.5-flash__12,1.0,1.0,1.0,1.0
qwen3-235b-a22b__12,1.0,1.0,1.0,1.0
deepseek-chat-v3-0324__12,1.0,1.0,1.0,1.0
llama-4-maverick__12,1.0,1.0,1.0,1.0


In [114]:
combined_df = pd.concat([df, df1])

In [115]:
combined_df

,оценка перевода (Иван),оценка перевода (Екатерина),оценка перевода (Артем),overall_person
gpt-4o__15,1.000000,0.175658,0.001253,0.999996
claude-sonnet-4__15,1.000000,0.387769,0.005886,1.000000
gemini-2.5-flash__15,0.999996,0.118529,0.000040,0.999956
qwen3-235b-a22b__15,0.999993,0.160640,0.000288,0.999976
deepseek-chat-v3-0324__15,0.999997,0.101087,0.000426,0.999987
llama-4-maverick__15,1.000000,0.214498,0.000771,0.999997
gpt-4o__12,1.000000,1.000000,1.000000,1.000000
claude-sonnet-4__12,1.000000,1.000000,1.000000,1.000000
gemini-2.5-flash__12,1.000000,1.000000,1.000000,1.000000
qwen3-235b-a22b__12,1.000000,1.000000,1.000000,1.000000
